## Beta_age Analysis

Calculate normalized beta_age values for TabulaMurisSenis mouse facs gene expression data. Modify counter to iterate through all 23 tissues. 

In [501]:
counter = 22

In [502]:
import pandas as pd
import math
import statistics
import numpy as np
import os
import operator as op

# Work with transcriptomic data (h5ad files)
import scanpy as sc # (conda environment connected to kernel)
import anndata
import h5py
import json

# Spearman and Pearson Correlation Rank and Linear Regression
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import linregress

Load analysis-expression configfile

In [503]:
configfile = "/global/home/users/pstein/Project-AgeExpressionConstraint/analysis/expression/config.json"
with open(configfile, "r") as f:
        config = json.load(f)

In [504]:
input_directory = config["TabulaMurisSenis"]
process_directory = config['beta_age']

In [505]:
tissues = []
tissue_files = []

for root, directories, filenames in os.walk(input_directory):
    for filename in filenames:
        # print(filename)
        tissue_files.append(filename)
        tissues.append(filename.split('-')[-1].split('.')[0])
        fpath = os.path.join(root,filename)

In [506]:
tissue_file_name = tissue_files[counter]
tissue_file_name

'tabula-muris-senis-facs-processed-official-annotations-GAT.h5ad'

In [507]:
tissue = tissue_file_name.split('-')[-1].split('.')[0]
tissue

'GAT'

In [508]:
# AnnData object (X, obs, vars)
tissue_file = sc.read_h5ad(filename = input_directory + tissue_file_name)

/global/home/users/pstein/.conda/envs/my-env/lib/python3.10/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/global/home/users/pstein/.conda/envs/my-env/lib/python3.10/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [509]:
# make gene expression data a df
tissue_data = tissue_file.to_df()

### Filter and Normalize Age

In [510]:
tissue_file.obs['cell_ontology_class'].value_counts()

mesenchymal stem cell of adipose    1562
myeloid cell                         718
endothelial cell                     341
B cell                               316
CD4-positive, alpha-beta T cell      140
CD8-positive, alpha-beta T cell      129
T cell                                84
epithelial cell                       63
NK cell                               53
Name: cell_ontology_class, dtype: int64

1. Remove rows (cell types) with only 1 age category (cannot perform regression on one category).
2. Filter for two age categories: 3m (young) & 24m (aged), remove 18m

In [511]:
# Dictionary of cell types were there is only one age category (either before or filtering for 3m and 24m)
bad_cells = config["bad_cells"]

In [512]:
if op.contains(bad_cells, tissue) == True:
    filter_cell = tissue_file.obs['cell_ontology_class'].isin(bad_cells[tissue]) # check which cell types affected in tissue
    tissue_data_filtered_cell = tissue_data[~filter_cell] # subset data to all but bad celltype
    tissue_meta_filtered_cell = tissue_file.obs[~filter_cell] # subset metadata to all but bad celltype
    filter_age = tissue_meta_filtered_cell['age'] != '18m' # remove all 18m aged rows
    tissue_data_filtered_age = tissue_data_filtered_cell[filter_age] 
    tissue_meta_filtered_age = tissue_meta_filtered_cell[filter_age]
else:
    filter_age = tissue_file.obs['age'] != '18m'
    tissue_data_filtered_age = tissue_data[filter_age]
    tissue_meta_filtered_age = tissue_file.obs[filter_age]

3. norm age data: 3m -> 0, 24m (&21m) -> 1

In [513]:
# Dict for norming age categories 
age_norm = {'3m': 0, '21m': 1, '24m': 1}

In [514]:
tissue_meta_filtered_age['age_norm'] = tissue_meta_filtered_age.loc[:,'age']

/tmp/ipykernel_170894/1281253903.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tissue_meta_filtered_age['age_norm'] = tissue_meta_filtered_age.loc[:,'age']


In [515]:
tissue_meta_filtered_age = tissue_meta_filtered_age.replace({"age_norm": age_norm})

In [516]:
tissue_meta_filtered_age.groupby("age_norm").size()

age_norm
0      1464
18m       0
1      1067
dtype: int64

### Automated beta_age Analysis 

1. Standardize gene expression data: remove all genes with no gene expression values across all samples
2. Z-score normalize genexpression data per gene 
3. Run linear regression and pearson correlation for age vs expression

In [517]:
gene_names = []
cell_types = []
rho = []
pval = []
linreg_slope = []

In [ ]:
for cell in tissue_meta_filtered_age['cell_ontology_class'].unique(): 
    ## subset data for the desired cell type 
    # subset cell type in metadata
    cell_type = tissue_meta_filtered_age['cell_ontology_class'] == cell
    # get age data
    cell_type_age = np.array(tissue_meta_filtered_age[cell_type]['age_norm'])
    # subset cell type in gene expression data 
    cell_type_data = tissue_data_filtered_age[cell_type]

    ## standardize 
    cell_type_data = cell_type_data.loc[:, (cell_type_data.sum(axis=0) != 0)]
    # z-score normalize
    cell_type_data = (cell_type_data - cell_type_data.mean()) / cell_type_data.std()
    # remove genes with nan z-score normalization because all gene expression values are exactly the same
    cell_type_data = cell_type_data.loc[:,~cell_type_data.isnull().any()]
    
    ## get regression data
    # itterate through each subset df
    for gene in np.array(cell_type_data.columns):
        cell_types.append(cell)
        gene_names.append(gene)
        pearson = pearsonr(cell_type_age.astype(float), np.array(cell_type_data[gene]))
        rho.append(pearson[0])
        regression = linregress(cell_type_age.astype(float), np.array(cell_type_data[gene]))
        linreg_slope.append(regression[0])
        pval.append(regression[3])

Format Data

In [ ]:
tissue_analysis = pd.DataFrame(
    {
        'gene': gene_names,
        'tissue': [tissue] * len(gene_names),
        'cell_type': cell_types,
        'rho': rho,
        'beta': linreg_slope,
        'p_value': pval
        }
    )

In [ ]:
tissue_analysis.head()

In [ ]:
# extension if normalizing data
filter_condition = '_age_expression_normed'

In [ ]:
tissue_csv = tissue_analysis.to_csv(process_directory + tissue + filter_condition + '.csv')